In [1]:
# MONDAY 3~5PM 
import pymysql,pandas as pd
conn = pymysql.connect(host='192.168.0.50', port=3306, user='eums', password='eums00!q', db='eums-cj-business', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute("select * from MEUMS_COMP_REVISIT_HIS WHERE DEAL_DT BETWEEN '2018-02-21 15:00:00' AND '2018-02-21 17:00:00';")
comp_p = pd.DataFrame( curs.fetchall() )[['EPOPCON_ID','COMPANY_ID','DEAL_DT','CARD_NAME','PAYMENT']]
conn.close()
comp_p.head()

,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT
0,cjcis15a39a8fa71e42756fbe8c5fc,6032218,2018-02-21 16:00:41,농협카드,835600
1,cjcis15a38f3eab267560d56e31b00,7461611,2018-02-21 16:23:44,하나카드(6*5*),3500
2,cjcis15a39ee7c71e42756fbe96c0e,5233223,2018-02-21 15:34:04,KB국민카드(4*7*),20062
3,cjcis15a751f3fe946d9036136c359,4919000,2018-02-21 15:00:33,현대카드,15400
4,cjcis15a3925c26259ad0a9e326589,7776647,2018-02-21 15:38:02,우리카드(7165),224074


In [2]:
print(comp_p.shape)

(81571, 5)


In [3]:
#STARBUCKS CUSTOMER
conn = pymysql.connect(host='192.168.0.50', port=3306, user='eums', password='eums00!q', db='eums-shared', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute("select ID, CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD, LATITUDE, LONGITUDE, UPT_DT from MEUMS_COMPANY WHERE ID IN "+str(tuple(comp_p.COMPANY_ID.unique()))+";")
starbucks = pd.DataFrame( curs.fetchall() )
conn.close()
print(starbucks.shape)
starbucks.head()

(32280, 10)


,ADDR,CATE,CATE1,CO_NAME,ID,LATITUDE,LONGITUDE,SEARCH_KEYWORD,TAG,UPT_DT
0,서울 중구 장충동2가 208,취미/스포츠,스포츠시설,서울클럽,207,37.5532149,127.003519,식당,"#스포츠, #스포츠센터, #레저, #스포츠시설",2018-03-21 16:46:39
1,서울특별시 중구 장충동2가 208,외식,외식,(사단)서울클럽,208,37.5532549,127.0038366,식당,"#한식, #음식점, #단체, #식당, #푸드, #관공서",2018-03-21 16:46:40
2,전남 영광군 홍농읍 상하리 567-17,마트,슈퍼마켓,지마트홍농점,748,35.3913859,126.4419573,마트,"#생활, #가정, #슈퍼마켓",2018-03-21 16:46:44
3,전북 전주시 완산구 평화동2가 895-1,마트,슈퍼마켓,자연에마트 전주점,766,35.7863775,127.1332331,마트,"#생활, #가정, #슈퍼마켓",2018-03-21 16:46:43
4,인천 남구 주안동 1534-4,의료/건강,병원,(의료법인)아인의료재단,917,37.4515386,126.6894403,병원,"#생활, #병원부속시설, #의료, #건강, #병원",2018-03-21 16:46:45


In [4]:
starbucks['COMPANY_ID']=starbucks['ID']
del starbucks['ID']

In [5]:
train = starbucks[starbucks['CO_NAME'].str.contains('스타벅스')]
print(train.shape)
train.head()

(344, 10)


,ADDR,CATE,CATE1,CO_NAME,LATITUDE,LONGITUDE,SEARCH_KEYWORD,TAG,UPT_DT,COMPANY_ID
5891,서울특별시 영등포구 문래동3가 77-1,카페/디저트,카페,스타벅스 문래동3가점,37.5196403,126.8896793,식당,"#테이크아웃커피, #카페, #베이커리, #식당, #디저트",2017-07-10 20:47:38,561640
5892,서울특별시 강남구 대치동 942-5,카페/디저트,카페,스타벅스 대치동점,37.5057896,127.0579569,식당,"#테이크아웃커피, #카페, #베이커리, #식당, #디저트",2017-07-10 20:47:38,561643
5893,서울 강남구 역삼동 679-1 GS타워 지하 1층,카페/디저트,카페,스타벅스 GS타워점,37.50179507577058,127.03707168520819,식당,"#스타벅스, #커피전문점, #음식점, #카페",2017-09-27 18:14:02,561652
5894,서울 송파구 가락동 79 롯데캐슬 파인힐1층,카페/디저트,카페,스타벅스 가락시장역점,37.494404078688554,127.12163916366208,식당,"#스타벅스, #커피전문점, #음식점, #카페",2017-10-03 15:31:08,561655
5895,서울 강서구 가양3동 449-19 가양이마트점 1층,카페/디저트,카페,스타벅스 가양이마트점,37.557629948645236,126.86264433331407,식당,"#스타벅스, #커피전문점, #음식점, #카페",2017-10-08 13:27:41,561660


In [6]:
test = pd.merge(comp_p, train, how = 'inner' , on='COMPANY_ID')
test =test[['EPOPCON_ID','COMPANY_ID','DEAL_DT','CARD_NAME','PAYMENT','CO_NAME','CATE','CATE1','TAG','ADDR','SEARCH_KEYWORD', 'LATITUDE', 'LONGITUDE']]#'UPT_DT'
print(test.shape)
test.head()

(731, 13)


,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD,LATITUDE,LONGITUDE
0,cjcis15a38bf14b267560d56e2d743,7969071,2018-02-21 15:00:18,신한카드(5*7*),10000,스타벅스 강남오거리점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 강남구 역삼동 813-17,None,37.50211503569852,127.02667182889539
1,cjcis15a8cffbe5a93f40fafc7bcbf,561829,2018-02-21 15:00:26,하나카드(7*8*),9200,스타벅스 발산역점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 강서구 등촌동 678-14 1층,식당,37.56006173054942,126.83909837972934
2,cjcis15a397dd7e946d9170e4c32b5,7486322,2018-02-21 15:00:44,하나카드(8*4*),15300,스타벅스 연세세브란스점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 중구 남대문로5가 84-11,None,37.55720592495911,126.97353946775377
3,cjcis15a392cf36259ad0a9e32834a,5052756,2018-02-21 15:01:14,현대카드(6205),17800,스타벅스 신설동역점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 동대문구 신설동 76-10,None,37.57662438578771,127.02556632839254
4,cjcis15a8269a85a93f40fafc78932,6764996,2018-02-21 15:00:48,현대카드,9700,스타벅스 한국프레스센터점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 중구 태평로1가 25,None,37.56718984408174,126.97779192263211


In [7]:
#test.to_csv('starbucks.csv')
#25명
count_id = pd.DataFrame(test['EPOPCON_ID'].value_counts())
print(count_id.shape)
count_id.head()
#test.nlargest(10,'EPOPCON_ID')
#test.sort('EPOPCON_ID', ascending=False)


(682, 1)


,EPOPCON_ID
cjcis15a6bb6bfb2675635e2576e88,9
cjcis15a38f181e946d9170e4afa6e,6
cjcis15a6bd4b2b2675635e2576f25,4
cjcis15a398a675a93f40965bd13fd,3
cjcis15a3ddac55a93f41b931d0ad1,3


# 결제건 2번이상인 사람
1. 결제취소 데이터 가능성. cjcis15a39353a5a93f40965bc8b24 
2. 한명이 여러명을 대신 결제하는 성향 보임
 - 한명이 하나의 카드로 결제한다. cjcis15a38eaf55a93f40965bbad5a
 - 한명이 다른 카드로 결제한다.  cjcis15a3d91c1f652660632fcb5c9  cjcis15a39016971e42756fbe70655
  +) 법인카드와 본인카드 결제하는 사람 발견. cjcis15a3944f55a93f40965bcaff2 
    
3. 시간차를 두고 여러번 시켜먹는 사람이 있음 
 - 1,2분차이로 결제하는 사람 cjcis15a3a6205e946d90511c9881d
 - 5분이상의 차이로 결제하는 사람 cjcis15a3933b371e42756fbe7c4a2

# 결제단위
1. 일만원 결제하는 사람
 - 기프트카드 구매하는 사람인가보다../수원성균관대점, 새문안로점.)
 - 법인카드 한도액이 일만원인가

In [8]:
count_id_one = count_id[count_id['EPOPCON_ID']<2]
print(count_id_one.shape)
countid_one = pd.DataFrame(count_id_one.reset_index()['index'])
countid_one['EPOPCON_ID']=countid_one['index']
countid_one.head()

(652, 1)


,index,EPOPCON_ID
0,cjcis15a399f046259ad0a9e337ad2,cjcis15a399f046259ad0a9e337ad2
1,cjcis15a39caffb267560d56e56d79,cjcis15a39caffb267560d56e56d79
2,cjcis15a39822e71e42756fbe83bdd,cjcis15a39822e71e42756fbe83bdd
3,cjcis15a3a6e81fec3e50db118354d,cjcis15a3a6e81fec3e50db118354d
4,cjcis15a3a1b43b267561f3d744c36,cjcis15a3a1b43b267561f3d744c36


In [9]:
user_cnt_one = pd.merge(countid_one,comp_p,how='inner', on = 'EPOPCON_ID')[['EPOPCON_ID','COMPANY_ID','DEAL_DT','CARD_NAME','PAYMENT']]
user_cnt_one = user_cnt_one.sort_values(['EPOPCON_ID','DEAL_DT'])
print(user_cnt_one.shape)
user_cnt_one.head()

(882, 5)


,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT
222,cjcis15a38a7f2b267560d56e2ccb0,561797,2018-02-21 15:50:47,씨티카드(4*9*),50000
674,cjcis15a38b49371e42756fbe6a0a1,8247792,2018-02-21 16:15:33,하나카드(0*7*),11600
675,cjcis15a38b49371e42756fbe6a0a1,562044,2018-02-21 16:23:33,하나카드(0*7*),4700
699,cjcis15a38bf14b267560d56e2d743,7969071,2018-02-21 15:00:18,신한카드(5*7*),10000
135,cjcis15a38c5ac71e42756fbe6ae27,6764996,2018-02-21 15:29:58,SC은행BC(8119),4000


# 1517시간대에 스타벅스에서 1번만 결제한 사람들은 1517시간대에 어떤 다른 결제를 했나?

len(user_cnt_one.COMPANY_ID.unique()) # 478명

#EXPLORE OTHER PAYMENT DATA 
conn = pymysql.connect(host='192.168.0.50', port=3306, user='eums', password='eums00!q', db='eums-shared', charset='utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)
curs.execute("select ID, CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD, LATITUDE, LONGITUDE, UPT_DT from MEUMS_COMPANY WHERE ID IN "+str(tuple(user_cnt_one.COMPANY_ID.unique()))+";")
exc_starbucks = pd.DataFrame( curs.fetchall() )
conn.close()
print(exc_starbucks.shape)
exc_starbucks.head()

exc_starbucks['COMPANY_ID']=exc_starbucks['ID']
del exc_starbucks['ID']
exc_starbucks.head()

In [10]:
pay_exc_starbucks = pd.merge(user_cnt_one, starbucks, how = 'inner' , on='COMPANY_ID')
pay_exc_starbucks =pay_exc_starbucks[['EPOPCON_ID','COMPANY_ID','DEAL_DT','CARD_NAME','PAYMENT','CO_NAME','CATE','CATE1','TAG','ADDR','SEARCH_KEYWORD', 'LATITUDE', 'LONGITUDE']]#'UPT_DT'
print(pay_exc_starbucks.shape)
pay_exc_starbucks.head()

(848, 13)


,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD,LATITUDE,LONGITUDE
0,cjcis15a38a7f2b267560d56e2ccb0,561797,2018-02-21 15:50:47,씨티카드(4*9*),50000,스타벅스 동성로중앙점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",대구 중구 삼덕동1가 17-11,식당,35.86749098666445,128.59665665952667
1,cjcis15a38b49371e42756fbe6a0a1,562044,2018-02-21 16:23:33,하나카드(0*7*),4700,스타벅스 종로평창점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 종로구 평창동 310-1 1~2층,은행,37.60580742025601,126.96678617461968
2,cjcis15a38bf14b267560d56e2d743,7969071,2018-02-21 15:00:18,신한카드(5*7*),10000,스타벅스 강남오거리점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 강남구 역삼동 813-17,None,37.50211503569852,127.02667182889539
3,cjcis15a38c5ac71e42756fbe6ae27,6764996,2018-02-21 15:29:58,SC은행BC(8119),4000,스타벅스 한국프레스센터점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 중구 태평로1가 25,None,37.56718984408174,126.97779192263211
4,cjcis15a38ce0d71e42756fbe6b5cb,6764996,2018-02-21 15:33:55,신한체크(7*1*),4100,스타벅스 한국프레스센터점,카페/디저트,카페,"#스타벅스, #커피전문점, #음식점, #카페",서울 중구 태평로1가 25,None,37.56718984408174,126.97779192263211


In [11]:
pay_exc_starbucks_copy = pay_exc_starbucks

In [12]:
train1 = pay_exc_starbucks[~pay_exc_starbucks['CO_NAME'].str.contains('스타벅스')]
print(train1.shape)
train1.head()

(196, 13)


,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD,LATITUDE,LONGITUDE
230,cjcis15a38cdd8e946d9170e4ac95a,1062463,2018-02-21 16:21:00,신한카드(0*9*),12990,홈플러스 화성동탄점,마트,대형마트,"#홈플러스, #생활, #가정, #대형마트",경기 화성시 반송동 98,마트,37.20231865462592,127.06871283823443
240,cjcis15a38dfbd6259ad0a9e31aaab,6631065,2018-02-21 15:34:07,하나카드(3*2*),5500,버거킹 포항법원앞SK점,외식,패스트푸드,"#버거킹, #패스트푸드, #음식점",경북 포항시 북구 장성동 1370-8 SK주유소 내,None,36.08734687049524,129.38812397239025
241,cjcis15a38e00a5a93f40965bb9e0d,571762,2018-02-21 16:57:42,현대카드,3100,시티프라자약국,의료/건강,약국,"#약국, #의료, #건강",서울 구로구 오류동 44-10,약국,37.4944725,126.8410109
251,cjcis15a38ed31b267560d56e311ee,7982902,2018-02-21 16:33:49,우리카드(5539),3800,롯데리아 롯데마트서울양평점,외식,패스트푸드,"#롯데리아,#외식,#패스트푸드,#햄버거,#음식점",서울 영등포구 양평동3가 45,None,37.52606742169527,126.89157724198161
252,cjcis15a38eddae946d9170e4af576,5704236,2018-02-21 15:43:25,현대카드,5500,김밥천국 삼선교점,외식,분식,"#분식, #김밥천국, #음식점",서울 성북구 동소문동1가 92,None,37.589518304372994,127.00789859168248


In [13]:
#train1.iloc[10,:]['PAYMENT'].astype('int')
#pd.to_numeric(train1['PAYMENT'],errors='coerce')
#type(.iloc[1,:])
print(train1[train1['PAYMENT'].astype('int')>0].shape)
train1[train1['PAYMENT'].astype('int')>0].head()
#aaa.head()

(194, 13)


,EPOPCON_ID,COMPANY_ID,DEAL_DT,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,TAG,ADDR,SEARCH_KEYWORD,LATITUDE,LONGITUDE
230,cjcis15a38cdd8e946d9170e4ac95a,1062463,2018-02-21 16:21:00,신한카드(0*9*),12990,홈플러스 화성동탄점,마트,대형마트,"#홈플러스, #생활, #가정, #대형마트",경기 화성시 반송동 98,마트,37.20231865462592,127.06871283823443
240,cjcis15a38dfbd6259ad0a9e31aaab,6631065,2018-02-21 15:34:07,하나카드(3*2*),5500,버거킹 포항법원앞SK점,외식,패스트푸드,"#버거킹, #패스트푸드, #음식점",경북 포항시 북구 장성동 1370-8 SK주유소 내,None,36.08734687049524,129.38812397239025
241,cjcis15a38e00a5a93f40965bb9e0d,571762,2018-02-21 16:57:42,현대카드,3100,시티프라자약국,의료/건강,약국,"#약국, #의료, #건강",서울 구로구 오류동 44-10,약국,37.4944725,126.8410109
251,cjcis15a38ed31b267560d56e311ee,7982902,2018-02-21 16:33:49,우리카드(5539),3800,롯데리아 롯데마트서울양평점,외식,패스트푸드,"#롯데리아,#외식,#패스트푸드,#햄버거,#음식점",서울 영등포구 양평동3가 45,None,37.52606742169527,126.89157724198161
252,cjcis15a38eddae946d9170e4af576,5704236,2018-02-21 15:43:25,현대카드,5500,김밥천국 삼선교점,외식,분식,"#분식, #김밥천국, #음식점",서울 성북구 동소문동1가 92,None,37.589518304372994,127.00789859168248


In [14]:
train1.CATE1.value_counts()

백화점      14
카페       13
편의점      11
온라인쇼핑    11
병원       10
베이커리     10
약국       10
대형마트      9
한식        7
택시        7
화장품       7
패스트푸드     6
공공기관      5
생활용품      5
슈퍼마켓      4
편의        3
간편결제      3
          3
서비스       3
건강식품      3
주유소       3
분식        3
휴대폰       2
제조업       2
학원        2
보험        2
헤어        2
기차        2
인터넷       2
시장        2
문화센터      2
중식        2
서점        2
버스        2
종교        1
어린이집      1
문화        1
부동산       1
건설        1
사무용품      1
전체        1
전화        1
명소        1
신발        1
영화        1
원예/조경     1
언론        1
운송/배송     1
골프        1
안경        1
일식        1
대리점       1
가전용품      1
아울렛       1
의류        1
동남아음식     1
Name: CATE1, dtype: int64